# Using a pretrained model - Inception v3

**Note:** We highly recommend running this notebook on a GPU. 

## 0. Initialization

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import os

os.chdir("..")
import requests
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from src.utils import seed_everything
from torchvision.models import inception_v3, Inception_V3_Weights
from src.loading import load_data
from src.train import train

In [ ]:
# Set the seeds
seed_everything()

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



---

<a name='s1'></a>
## 1. Downloading the dataset

Fetching the dataset should take around 4-5 minutes. Unzipping takes 20s.

In [ ]:
# if folder 'data/' is does not exist, download the data
if not os.path.exists("data/"):
    # Dropbox URL
    dropbox_url = "https://www.dropbox.com/scl/fi/sa14unf8s47e9ym125zgo/data.zip?rlkey=198bg0cmbmmrcjkfufy9064wm&dl=1"

    # File path where the .zip file will be saved
    file_path = "data.zip"

    response = requests.get(dropbox_url)

    if response.status_code == 200:
        with open(file_path, "wb") as file:
            file.write(response.content)
        message = "Download successful. The file has been saved as 'data.zip'."
    else:
        message = "Failed to download the file. Error code: " + str(
            response.status_code
        )

    print(message)

    # Path to the downloaded .zip file
    zip_file_path = "data.zip"

    # Directory to extract the contents of the zip file
    extraction_path = ""

    # Unzipping the file
    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extractall(extraction_path)

    extraction_message = (
        f"The contents of the zip file have been extracted to: {extraction_path}"
    )

    print(extraction_message)

## 2. Data

Load the data with improved preprocessing. Set the batch size according to your machine, here we tried to set it as high as possible, as long as the GPU has enough memory.

In [ ]:
# Load the .jpeg files in the data folder
PATH_IMAGES = "data/images_keep_ar"
PATH_LABELS = "data/labels/trainLabels.csv"
batch_size = 8  # default 8
img_size = (300, 300)  # default size 120/120
num_epochs = 20

Load train and validation, 90-10 ratio. 

In [ ]:
train_loader, validation_loader = load_data(
    PATH_LABELS, PATH_IMAGES, img_size, batch_size
)

In [ ]:
# Visualize an image
for images, labels in train_loader:
    print(images.shape)
    print(labels.shape)
    plt.figure(figsize=(4, 4))
    plt.axis("off")
    plt.imshow(np.transpose(images[0], (1, 2, 0)))
    plt.show()
    break

## 3. Model

Here we are gonna use the InceptionV3 model. We will use the pretrained weights from the ImageNet dataset. We will freeze the weights of the model and only train the last few layers.
Inception_V3 has auxiliary outputs that are not used in this notebook. Inception v3 consists of five convolutional layers, two max-pooling layers, 11 inception modules, one average pooling layer, and one fully connected layer.
We also alter the last layer to fit our problem, since we have 5 classes instead of 1000.

In [ ]:
# Initialize model with the best available weights
weights = Inception_V3_Weights.DEFAULT
model = inception_v3(weights=weights)

We will unfreeze `InceptionE` layer, in order to fine-tune the model.

To see the structure: `print(model)`

In [ ]:
parameters_to_train = []

# Freeze all the layers in the network
for param in model.parameters():
    param.requires_grad = False

# Replace the last layer
model.fc = nn.Linear(2048, 5)

# Unfreeze layer 7
for param in model.fc.parameters():
    param.requires_grad = True
    parameters_to_train.append(param)

for param in model.Mixed_7c.parameters():
    param.requires_grad = True
    parameters_to_train.append(param)

for param in model.Mixed_7b.parameters():
    param.requires_grad = True
    parameters_to_train.append(param)

In [ ]:
optimizer = optim.Adam(parameters_to_train, lr=3e-5)
criterion = nn.CrossEntropyLoss()
model.to(device)

Optional. If you want to fine tune this model, we advise you to do in in steps. Load the last checkpoint to continue with training. 

In [ ]:
# MODEL = "results/models/inception_baseline.pt"
# checkpoint = torch.load(MODEL)
# model.load_state_dict(checkpoint)

### 3.1 Model training and evaluation

In [ ]:
train(
    model,
    train_loader,
    validation_loader,
    optimizer,
    criterion,
    device,
    num_epochs,
    "results/models/inception_baseline.pt",
)